In [1]:
import numpy as np
import pandas as pd

In [2]:
pathslist = ['/Users/plutzer/Box/CellBio-MajorLab/Separate PG files/1-22-2021-MA-ADCK1_PG.txt',
'/Users/plutzer/Box/CellBio-MajorLab/Separate PG files/1-29-2021-MA-CSNK1G1_PG.txt',
'/Users/plutzer/Box/CellBio-MajorLab/Separate PG files/1-29-2021-MA-CSNK1G2_PG.txt',
'/Users/plutzer/Box/CellBio-MajorLab/Separate PG files/1-29-2021-MA-CSNK1G3_PG.txt',
'/Users/plutzer/Box/CellBio-MajorLab/Separate PG files/1-25-2021-MA-VRK2_PG.txt']

In [3]:
starting_df = pd.read_csv(pathslist[0],delimiter = '\t')

In [4]:
new_df = pd.read_csv(pathslist[1],delimiter = '\t')

In [66]:
def merge_pgs(dataframe1,dataframe2 = None):
    '''
        Merges two proteinGroups files on simplified Protein IDs, and rectifies other columns needed for downstream analysis.
            If no second dataframe is given, the function will perform filtering and column simplification steps on just the 
            first dataframe. Otherwise, the first dataframe is assumed to already be rectified and the filtering/simplification
            will only be done on the second dataframe.
        dataframe1: pandas dataframe for first proteinGroups file
        dataframe2: pandas dataframe for second proteinGroups file. Default is null if just the first dataframe is being filtered. 
        Output: pandas dataframe for the merged proteinGroups file, or filtered dataframe.
    '''
    # If no second dataframe is given, just filter and fix the Protein IDs column of the first dataframe.
    if dataframe2 is None:
        # Use the simplify function to fix the Protein IDs column.
        dataframe1 = simplify_protein_IDs(dataframe1)
        #Filter out any rows where the protein group was identified only by type
        dataframe1 = dataframe1[dataframe1['Score'] > 0]
        # Return the filtered and fixed dataframe
        return dataframe1
    # If a second dataframe is given, it is assumed that the first dataframe is already filtered and rectified.
    else:
        # Simplify the Protein IDs column.
        dataframe2 = simplify_protein_IDs(dataframe2)
        #Filter out any rows where the protein group was identified only by type
        dataframe2 = dataframe2[dataframe2['Score'] > 0]
        
        # Merge the columns unique to the second dataframe with the first dataframe using protein IDs
        combined_df = dataframe1.merge(dataframe2[list(np.setdiff1d(dataframe2.columns,dataframe1.columns))+['Protein IDs']],how='outer',on='Protein IDs')
        
        ##### Next fix the columns of the new dataframe. I'm only fixing the columns that matter for downstream SAINT analysis.
        
        # Fix Score (max of scores for a given protein group)
        max_scores = []
        for group in combined_df['Protein IDs']:
            max_scores.append(np.max(list(dataframe1[dataframe1['Protein IDs'] == group]['Score'])+list(dataframe2[dataframe2['Protein IDs'] == group]['Score'])))
        combined_df['Score'] = max_scores
        
        # Fix Reverse (+ if either dataframe has a +)
        reverse = []
        for group in combined_df['Protein IDs']:
            if ('+' in list(dataframe1[dataframe1['Protein IDs'] == group]['Reverse'])) or ('+' in list(dataframe2[dataframe2['Protein IDs'] == group]['Reverse'])):
                reverse.append('+')
            else:
                reverse.append('')
        combined_df['Reverse'] = reverse
        
        # Fix Contaminant (+ if either dataframe has a +)
        contaminant = []
        for group in combined_df['Protein IDs']:
            if ('+' in list(dataframe1[dataframe1['Protein IDs'] == group]['Potential contaminant'])) or ('+' in list(dataframe2[dataframe2['Protein IDs'] == group]['Potential contaminant'])):
                contaminant.append('+')
            else:
                contaminant.append('')
                
        # Fix Sequence length (should be the same - still use max just in case)
        max_seqlength = []
        for group in combined_df['Protein IDs']:
            max_seqlength.append(np.max(list(dataframe1[dataframe1['Protein IDs'] == group]['Sequence length'])+list(dataframe2[dataframe2['Protein IDs'] == group]['Sequence length'])))
        combined_df['Sequence length'] = max_seqlength
        # Fix Identified by site (+ if either dataframe has a + ... just in case: shouldn't matter because I'm filtering these out to start)
        combined_df['Only identified by site'] = ['' for item in range(len(combined_df))]
        
        # Return the combined dataframe
        return combined_df

    

In [15]:
def simplify_protein_IDs(dataframe):
    '''
        Takes a proteinGroups dataframe and returns a proteinGroups dataframe with simplified protein IDs.
        dataframe: pandas Dataframe with a 'Protein IDs' column
        Output: pandas Dataframe with the 'Protein IDs' column simplified
    '''
    dataframe['Protein IDs'] = [protein_group.split(';')[0] for protein_group in dataframe['Protein IDs']]
    return dataframe

In [67]:
combed_df = merge_pgs(merge_pgs(starting_df),new_df)

In [69]:
combed_df.columns

Index(['Protein IDs', 'Majority protein IDs', 'Peptide counts (all)',
       'Peptide counts (razor+unique)', 'Peptide counts (unique)',
       'Protein names', 'Gene names', 'Fasta headers', 'Number of proteins',
       'Peptides', 'Razor + unique peptides', 'Unique peptides',
       'Peptides ADCK1_1', 'Peptides ADCK1_2',
       'Razor + unique peptides ADCK1_1', 'Razor + unique peptides ADCK1_2',
       'Unique peptides ADCK1_1', 'Unique peptides ADCK1_2',
       'Sequence coverage [%]', 'Unique + razor sequence coverage [%]',
       'Unique sequence coverage [%]', 'Mol. weight [kDa]', 'Sequence length',
       'Sequence lengths', 'Fraction average', 'Fraction 1', 'Q-value',
       'Score', 'Identification type ADCK1_1', 'Identification type ADCK1_2',
       'Sequence coverage ADCK1_1 [%]', 'Sequence coverage ADCK1_2 [%]',
       'Intensity', 'Intensity ADCK1_1', 'Intensity ADCK1_2',
       'LFQ intensity ADCK1_1', 'LFQ intensity ADCK1_2', 'MS/MS count ADCK1_1',
       'MS/MS count 

In [20]:
testdf = starting_df.merge(new_df[list(np.setdiff1d(new_df.columns,starting_df.columns))+['Protein IDs']],how='outer',on='Protein IDs')
testdf

,Protein IDs,Majority protein IDs,Peptide counts (all),Peptide counts (razor+unique),Peptide counts (unique),Protein names,Gene names,Fasta headers,Number of proteins,Peptides,...,MS/MS count CK1G1_1,MS/MS count CK1G1_2,Peptides CK1G1_1,Peptides CK1G1_2,Razor + unique peptides CK1G1_1,Razor + unique peptides CK1G1_2,Sequence coverage CK1G1_1 [%],Sequence coverage CK1G1_2 [%],Unique peptides CK1G1_1,Unique peptides CK1G1_2
0,Q9NZJ9,Q9NZJ9;A0A024RBG1,1;1,1;1,1;1,NaN,NUDT4;NUDT4B,sp|Q9NZJ9|NUDT4_HUMAN Diphosphoinositol polyph...,2.0,1.0,...,0.0,1.0,1.0,2.0,1.0,2.0,5.0,12.8,0.0,1.0
1,P0DPI2,P0DPI2;A0A0B4J2D5,4;4,4;4,4;4,NaN,GATD3A;GATD3B,sp|P0DPI2|GAL3A_HUMAN Glutamine amidotransfera...,2.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A0A0U1RRL7,A0A0U1RRL7,1,1,1,NaN,MMP24OS,sp|A0A0U1RRL7|MMPOS_HUMAN Protein MMP24OS OS=H...,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A0AVF1,A0AVF1,1,1,1,NaN,TTC26,sp|A0AVF1|IFT56_HUMAN Intraflagellar transport...,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A0AVT1,A0AVT1,1,1,1,NaN,UBA6,sp|A0AVT1|UBA6_HUMAN Ubiquitin-like modifier-a...,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4114,REV__Q9NQ89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0
4115,REV__Q9NXE4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0
4116,REV__Q9NXW9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,1.0,2.0,1.0,2.0,1.0,0.0,0.0,2.0,1.0
4117,REV__Q9P2T0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0


In [42]:
max_scores = []
for group in list(testdf['Protein IDs']):
    max_scores.append(np.max(list(starting_df[starting_df['Protein IDs'] == group]['Score'])+list(new_df[new_df['Protein IDs'] == group]['Score'])))
        

In [63]:
reverse = []
for group in list(testdf['Protein IDs']):
    #print('+' in list(starting_df[starting_df['Protein IDs'] == group]['Reverse']))
    if ('+' in list(starting_df[starting_df['Protein IDs'] == group]['Reverse'])) or ('+' in list(new_df[new_df['Protein IDs'] == group])):
        #print(''.join(starting_df[starting_df['Protein IDs'] == group]['Reverse'].astype('str')))
        reverse.append('+')
    else:
        reverse.append('')

In [52]:
list(testdf['Protein IDs'])

['Q9NZJ9',
 'P0DPI2',
 'A0A0U1RRL7',
 'A0AVF1',
 'A0AVT1',
 'A0FGR8',
 'A0MZ66',
 'A1KXE4',
 'A1L0T0',
 'A1X283',
 'A2RTX5',
 'A3KMH1',
 'A3KN83',
 'A5YKK6',
 'A6NDG6',
 'A6NED2',
 'A6NHQ2',
 'A6NHR9',
 'A6NJ78',
 'A6NKT7',
 'A8CG34',
 'A8K0Z3',
 'A8MTL9',
 'P62308',
 'Q99613',
 'CON__ENSEMBL:ENSBTAP00000024146',
 'CON__P00761',
 'P02533',
 'CON__P02666',
 'CON__P02769',
 'P48668',
 'P05787',
 'P07477',
 'P08779',
 'CON__P12763',
 'P13645',
 'P13647',
 'P35527',
 'P35908',
 'CON__Q6IME9',
 'CON__Streptavidin',
 'CON__tr|K7X1Z1|K7X1Z1_MYCHR',
 'CON__tr|K7X1Z5|K7X1Z5_MYCHR',
 'CON__tr|K7X232|K7X232_MYCHR',
 'CON__tr|K7X2F6|K7X2F6_MYCHR',
 'CON__tr|K7X2P4|K7X2P4_MYCHR',
 'CON__tr|K7X374|K7X374_MYCHR',
 'CON__tr|K7X390|K7X390_MYCHR',
 'CON__tr|K7X7W9|K7X7W9_MYCHR',
 'CON__tr|K7X804|K7X804_MYCHR',
 'CON__tr|K7X824|K7X824_MYCHR',
 'CON__tr|K7X838|K7X838_MYCHR',
 'CON__tr|K7X8J0|K7X8J0_MYCHR',
 'CON__tr|K7X8N3|K7X8N3_MYCHR',
 'CON__tr|K7X8U9|K7X8U9_MYCHR',
 'CON__tr|K7X8V3|K7X8V3_MYCHR',
 'CO

In [51]:
list(testdf['Reverse'])

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan

In [64]:
for group in combed_df['Protein IDs']:
    print(group)

Q9NZJ9
P0DPI2
A0A0U1RRL7
A0AVF1
A0AVT1
A0FGR8
A0MZ66
A1KXE4
A1L0T0
A1X283
A2RTX5
A3KMH1
A3KN83
A5YKK6
A6NDG6
A6NED2
A6NHQ2
A6NHR9
A6NJ78
A6NKT7
A8CG34
A8K0Z3
P62308
Q99613
CON__ENSEMBL:ENSBTAP00000024146
CON__P00761
P02533
CON__P02666
CON__P02769
P48668
P05787
P07477
P08779
CON__P12763
P13645
P13647
P35527
P35908
CON__Q6IME9
CON__Streptavidin
CON__tr|K7X1Z1|K7X1Z1_MYCHR
CON__tr|K7X1Z5|K7X1Z5_MYCHR
CON__tr|K7X2F6|K7X2F6_MYCHR
CON__tr|K7X2P4|K7X2P4_MYCHR
CON__tr|K7X374|K7X374_MYCHR
CON__tr|K7X390|K7X390_MYCHR
CON__tr|K7X7W9|K7X7W9_MYCHR
CON__tr|K7X804|K7X804_MYCHR
CON__tr|K7X824|K7X824_MYCHR
CON__tr|K7X838|K7X838_MYCHR
CON__tr|K7X8J0|K7X8J0_MYCHR
CON__tr|K7X8N3|K7X8N3_MYCHR
CON__tr|K7X8U9|K7X8U9_MYCHR
CON__tr|K7X8V6|K7X8V6_MYCHR
CON__tr|K7X924|K7X924_MYCHR
CON__tr|K7X928|K7X928_MYCHR
CON__tr|K7X973|K7X973_MYCHR
CON__tr|K7XA84|K7XA84_MYCHR
CON__tr|K7XKN1|K7XKN1_MYCHR
CON__tr|K7XKP6|K7XKP6_MYCHR
CON__tr|K7XLI1|K7XLI1_MYCHR
CON__tr|K7XLM4|K7XLM4_MYCHR
CON__tr|K7XLM7|K7XLM7_MYCHR
CON__tr|K7X

Q9Y3S1
Q9Y450
Q9Y484
Q9Y485
Q9Y496
Q9Y4D1
Q9Y4D8
Q9Y4E6
Q9Y4E8
Q9Y4F1
Q9Y4F5
Q9Y4G6
Q9Y4I1
Q9Y4J8
Q9Y4K4
Q9Y4P8
Q9Y4X5
Q9Y587
Q9Y597
Q9Y5B6
Q9Y5P4
Q9Y5X2
Q9Y5X3
Q9Y5Y0
Q9Y606
Q9Y618
Q9Y619
Q9Y624
Q9Y666
Q9Y672
Q9Y6D0
Q9Y6M4
Q9Y6M5
Q9Y6M7
Q9Y6N7
Q9Y6Q2
Q9Y6R0
Q9Y6Y0
REV__CON__tr|K7X9B6|K7X9B6_MYCHR
REV__CON__tr|K7XM01|K7XM01_MYCHR
REV__CON__tr|K7XYD1|K7XYD1_MYCHR
REV__O94966
REV__P42285
REV__P42898
REV__P78371
REV__Q08881
REV__Q13144
REV__Q14993
REV__Q6ZU52
REV__Q86TB9
REV__Q86UC2
REV__Q86V81
REV__Q86VH4
REV__Q8N2H3
REV__Q8N6R0
REV__Q8N8E1
REV__Q8N998
REV__Q8WUY3
REV__Q969R5
REV__Q96G01
REV__Q99418
REV__Q99797
REV__Q9C0G6
REV__Q9H410
REV__Q9NQ89
REV__Q9NXW9
REV__Q9P2T0
REV__Q9Y2I6


In [72]:
for item in range(1,10):
    print(item)

1
2
3
4
5
6
7
8
9
